In [6]:
import psycopg2
from tqdm.notebook import tqdm

# Database connection configuration
DB_CONFIG = {
    'dbname': 'medols',
    'user': 'postgres',
    'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    'host': '98.98.117.105',
    'port': '5432'
}

def connect_db():
    return psycopg2.connect(**DB_CONFIG)

def get_monitoring_ids():
    """
    Fetch all key_monitoring_media_online and key_monitoring_media_social values
    from the specified query.
    """
    query = """
    select 	d.id as key_monitoring_media_online,
            e.id as key_monitoring_media_social
    from 	monitoring a 
            inner join screen_analisis_ai b
            on cast(a.id as varchar) = b.monitoring_id
            inner join monitoring_search c 
            on b.monitoring_id = cast(c.monitoring_id as varchar)
            left outer join monitoring_media_online d 
                on d.monitoring_id = c.monitoring_id 
            left outer join monitoring_media_social e
                on e.monitoring_id = c.monitoring_id
    where b.status = 3 and b.active = 1 and b.jenis_analisa = '100'
    order by b.id desc 
    """
    try:
        conn = connect_db()
        with conn.cursor() as cur:
            cur.execute(query)
            # Fetch all results as a list of tuples
            monitoring_ids = cur.fetchall()
        return monitoring_ids  # Return list of tuples (key_monitoring_media_online, key_monitoring_media_social)
    except Exception as e:
        print(f"An error occurred while fetching monitoring_ids: {e}")
        return []
    finally:
        conn.close()

def run_insert_statements(monitoring_id):
    # Delete existing records for this monitoring_id
    delete_query = f"DELETE FROM social_media_activity WHERE monitoring_id_platform_data = '{monitoring_id}';"
    
    queries = [
        # Insert from vw_monitoring_facebook
        f"""
        INSERT INTO social_media_activity (
        monitoring_id_platform_data, name, description, created_date_platform_data,
        post_date, user_id_platform_data, profile_id, likes_count, shares_count,
        views_count, comment_count, sentiment_id, sentiment_category, sentiment_score,
        post_url, profile_url, cluster_no, cluster_score, pca_1, pca_2,
        iskol, img_cluster_no, img_pca_1, img_pca_2, img_cluster_score,
        table_name_platform_data, image_filename )
        SELECT monitoring_id, name, description, created_date, post_date,
        user_id, profile_id, likes_count, shares_count, views_count,
        comment_count, sentiment_id, sentiment_category, score AS sentiment_score,
        post_url, profile_url, cluster_no, cluster_score, pca_1, pca_2, iskol,
        img_cluster_no, img_pca_1, img_pca_2, img_cluster_score, 'vw_monitoring_facebook', filename
        FROM vw_monitoring_facebook
        WHERE monitoring_id = '{monitoring_id}';
        """,
        
        # Insert from vw_monitoring_google
        f"""
        INSERT INTO social_media_activity (
        monitoring_id_platform_data, source, description, created_date_platform_data,
        publish_date, updated_date_platform_data, link_url, content,
        title_platform_data, sentiment_category, sentiment_score, sentiment_id,
        cluster_no, cluster_score, pca_1, pca_2, img_cluster_no,
        img_cluster_score, img_pca_1, img_pca_2, table_name_platform_data)
        SELECT monitoring_id, source, description, created_date,
        publish_date, updated_date, link, content, title,
        sentiment_category, score AS sentiment_score, sentiment_id,
        cluster_no, cluster_score, pca_1, pca_2,
        img_cluster_no, img_cluster_score, img_pca_1,
        img_pca_2, 'vw_monitoring_google' AS table_name_platform_data
        FROM vw_monitoring_google
        WHERE monitoring_id = '{monitoring_id}';
        """,
        
        # Insert from vw_monitoring_instagram
        f"""
        INSERT INTO social_media_activity (
        instagram_post_id, link_url, post_content, publish_date,
        like_count, shares_count, comment_count, created_date_platform_data,
        monitoring_id_platform_data, profile_id, fullname,
        username_platform_data, is_tc_completed, post_exclude,
        sentiment_id, sentiment_category, sentiment_score, cluster_no,
        cluster_score, pca_1, pca_2, iskol, img_cluster_no,
        img_cluster_score, img_pca_1, img_pca_2, table_name_platform_data)
        SELECT instagram_post_id, link_url, post_content, publish_date,
        like_count, shared_count AS shares_count, comment_count,
        created_date, monitoring_id, profile_id, fullname,
        username, is_tc_completed, post_exclude, sentiment_id,
        sentiment_category, sentiment_score, cluster_no, cluster_score,
        pca_1, pca_2, iskol,img_cluster_no, img_cluster_score,
        img_pca_1, img_pca_2, 'vw_monitoring_instagram'
        FROM vw_monitoring_instagram
        WHERE monitoring_id = '{monitoring_id}';
        """,
        
        # Insert from vw_monitoring_tiktok
        f"""
        INSERT INTO social_media_activity (
        tiktok_id, monitoring_id_platform_data,create_time, description,
        is_polri, tiktok_created_date, tiktok_updated_date, nickname,
        unique_id, comment_count, digg_count, play_count, share_count,
        sentiment_category, sentiment_score, sentiment_id, follower_count,
        following_count, heart_count, video_count, cluster_no, cluster_score,
        pca_1, pca_2, iskol, img_cluster_no,
        img_pca_1, img_pca_2, img_cluster_score, table_name_platform_data,
        image_filename)
        SELECT tiktok_id, monitoring_id, create_time, "desc" AS description,
        is_polri, tiktok_created_date, tiktok_updated_date,
        nickname, unique_id, comment_count, digg_count,
        play_count, share_count, sentiment_category, score AS sentiment_score,
        id AS sentiment_id, follower_count, following_count, heart_count,
        video_count, cluster_no, cluster_score, pca_1, pca_2,
        iskol, img_cluster_no, img_pca_1, img_pca_2,
        img_cluster_score, 'vw_monitoring_tiktok', filename
        FROM vw_monitoring_tiktok
        WHERE monitoring_id = '{monitoring_id}';
        """,
        
        # Insert from vw_monitoring_twitter
        f"""
        INSERT INTO social_media_activity (
        twitter_tweets_id, monitoring_id_platform_data, user_id_platform_data, date,
        time,timezone,screen_name,name,is_polri,is_tc_completed,original_tweet,
        tweet,replies_count,likes_count,retweets_count,view_count,follower_count,
        sentiment_category,sentiment_score,sentiment_id,cluster_no,cluster_score,
        pca_1,pca_2,iskol,img_cluster_no,img_pca_1,img_pca_2, img_cluster_score,
        table_name_platform_data, image_filename )
        SELECT twitter_tweets_id, monitoring_id, user_id, date, time,
        timezone, screen_name, name, is_polri, is_tc_completed, original_tweet,
        tweet, replies_count, likes_count, retweets_count, view_count,
        follower_count, sentiment_category, score AS sentiment_score, sentiment_id,
        cluster_no, cluster_score, pca_1, pca_2, iskol, img_cluster_no,
        img_pca_1, img_pca_2, img_cluster_score, 'vw_monitoring_twitter', filename
        FROM vw_monitoring_twitter
        WHERE monitoring_id = '{monitoring_id}';
        """,
        
        # Insert from vw_monitoring_youtube
        f"""
        INSERT INTO social_media_activity (
        id_platform_data, monitoring_id_platform_data, channel_id, title_platform_data,
        description, channel_name, published_date, publish_time, created_date_platform_data,
        view_count, like_count, favorite_count, comment_count, sentiment_id,
        sentiment_category, sentiment_score, cluster_no, cluster_score,
        pca_1, pca_2, subscriber_count, video_count, channel_created_date,
        iskol,img_cluster_no,img_pca_1, img_pca_2, img_cluster_score, 
        table_name_platform_data)
        SELECT id, monitoring_id, channel_id, title, description, channel_name,
        published_date, publish_time, created_date, view_count, like_count,
        favorite_count, comment_count, sentiment_id, sentiment_category,
        score AS sentiment_score, cluster_no, cluster_score, pca_1,
        pca_2, subscriber_count, video_count, channel_created_date,
        iskol, img_cluster_no, img_pca_1, img_pca_2, img_cluster_score, 
        'vw_monitoring_youtube'
        FROM vw_monitoring_youtube
        WHERE monitoring_id = '{monitoring_id}';
        """
    ]

    rows_affected = []

    try:
        conn = connect_db()
        with conn.cursor() as cur:
            # Execute delete query
            cur.execute(delete_query)
            print(f"Deleted existing records for monitoring_id: {monitoring_id}")
            
            for query in queries:
                cur.execute(query)
                rows_affected.append(cur.rowcount)
        conn.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    finally:
        conn.close()

    return rows_affected

def main():
    monitoring_ids = get_monitoring_ids()  # Fetch all monitoring_ids

    for key_monitoring_media_online, key_monitoring_media_social in tqdm(monitoring_ids, desc="Processing Monitoring IDs"):
        # Process key_monitoring_media_online if not null
        if key_monitoring_media_online:
            print(f"Processing key_monitoring_media_online: {key_monitoring_media_online}")
            rows_inserted = run_insert_statements(key_monitoring_media_online)
            print(f"Rows affected for key_monitoring_media_online {key_monitoring_media_online}: {rows_inserted}")

        # Process key_monitoring_media_social if not null
        if key_monitoring_media_social:
            print(f"Processing key_monitoring_media_social: {key_monitoring_media_social}")
            rows_inserted = run_insert_statements(key_monitoring_media_social)
            print(f"Rows affected for key_monitoring_media_social {key_monitoring_media_social}: {rows_inserted}")

# Run the main function to start the process
main()


Processing Monitoring IDs:   0%|          | 0/624 [00:00<?, ?it/s]

Processing key_monitoring_media_online: 60d0a7bc-0c0b-4e96-8a92-55817369761e
Deleted existing records for monitoring_id: 60d0a7bc-0c0b-4e96-8a92-55817369761e
Rows affected for key_monitoring_media_online 60d0a7bc-0c0b-4e96-8a92-55817369761e: [0, 59, 0, 0, 0, 0]
Processing key_monitoring_media_social: acd58e91-f2f0-481a-9e41-6642fde848c9
Deleted existing records for monitoring_id: acd58e91-f2f0-481a-9e41-6642fde848c9
Rows affected for key_monitoring_media_social acd58e91-f2f0-481a-9e41-6642fde848c9: [0, 0, 0, 0, 28030, 0]
Processing key_monitoring_media_online: 6d4375a9-73c1-4f26-97a5-a615f7f5394b
Deleted existing records for monitoring_id: 6d4375a9-73c1-4f26-97a5-a615f7f5394b
Rows affected for key_monitoring_media_online 6d4375a9-73c1-4f26-97a5-a615f7f5394b: [0, 13, 0, 0, 0, 0]
Processing key_monitoring_media_social: 0941718c-56f6-441f-b145-11e9bef4c100
Deleted existing records for monitoring_id: 0941718c-56f6-441f-b145-11e9bef4c100
Rows affected for key_monitoring_media_social 0941718